# Decomposing unitary matrix into quantum gates

This tool is useful when you have $2^n \times 2^n$ matrix representing a untary operator acting on register of $n$ bits and want to implement this operator in Q#.

This notebook demonstrates how to use it.

## Tl;DR

In [2]:
import numpy as np
import quantum_decomp as qd

SWAP = np.matrix([[1,0,0,0],[0,0,1,0],[0,1,0,0], [0,0,0,1]])
print(qd.matrix_to_qsharp(SWAP))

operation ApplyUnitaryMatrix (qs : Qubit[]) : Unit {
body (...) {
    Controlled X([qs[1]], (qs[0]));
    Controlled X([qs[0]], (qs[1]));
    Controlled X([qs[1]], (qs[0]));
  }
}



## Example

Consider following matrix:

$$A = \frac{1}{\sqrt{3}}
\begin{pmatrix}
    1  & 1 & 1 & 0 \\
    1  & e^{\frac{2\pi i}{3}} & e^{\frac{4 \pi i}{3}} & 0 \\
    1  & e^{\frac{4\pi i}{3}} & e^{\frac{2 \pi i}{3}} & 0 \\
    0 & 0 & 0 & -i \sqrt{3} 
\end{pmatrix}$$

This is $3\times 3$ [DFT matrix](https://en.wikipedia.org/wiki/DFT_matrix), padded to have shape $4 \times 4$. Implementing such matrix was one way to solve problem B2 in [Microsoft Q# Coding Contest - Winter 2019](https://codeforces.com/blog/entry/65579).
[Here](https://assets.codeforces.com/rounds/1116/contest-editorial.pdf) you can find another approach to implementing this matrix, but let's see how we can implement it using our tool and Q#.

First, let's construct this matrix:

In [1]:
import numpy as np
w = np.exp((2j / 3) * np.pi)
A = np.array([[1, 1, 1, 0], 
                  [1, w, w * w, 0],
                  [1, w * w, w, 0], 
                  [0, 0, 0, -1j*np.sqrt(3)]]) / np.sqrt(3)
print(A)

[[ 0.57735027+0.j   0.57735027+0.j   0.57735027+0.j   0.        +0.j ]
 [ 0.57735027+0.j  -0.28867513+0.5j -0.28867513-0.5j  0.        +0.j ]
 [ 0.57735027+0.j  -0.28867513-0.5j -0.28867513+0.5j  0.        +0.j ]
 [ 0.        +0.j   0.        +0.j   0.        +0.j   0.        -1.j ]]


Now, let's use quantum_decomp library to construct Q# code.

In [2]:
import quantum_decomp as qd
print(qd.matrix_to_qsharp(A))

operation ApplyUnitaryMatrix (qs : Qubit[]) : Unit {
body (...) {
    Controlled X([qs[1]], (qs[0]));
    Controlled Ry([qs[0]], (-1.570796326794897, qs[1]));
    X(qs[1]);
    Controlled Ry([qs[1]], (-1.910633236249018, qs[0]));
    X(qs[1]);
    Controlled Rz([qs[0]], (-4.712388980384691, qs[1]));
    Controlled Ry([qs[0]], (-1.570796326794896, qs[1]));
    Controlled Rz([qs[0]], (-1.570796326794896, qs[1]));
    Controlled Rz([qs[1]], (-1.570796326794897, qs[0]));
    Controlled Ry([qs[1]], (-3.141592653589793, qs[0]));
    Controlled Rz([qs[1]], (1.570796326794897, qs[0]));
  }
}



As you can see from code in qsharp/ directory of this repository, this code indeed implements given unitary matrix. 

Also you can get the same sequence of operations as sequence of gates, where each gate is instance of GateFC or GateSingle, which are internal classes implementing fully controlled gate or gate acting on single qubit.

In [3]:
gates = qd.matrix_to_gates(A)
print('\n'.join(map(str, gates)))

X on bit 0, fully controlled
Ry(1.5707963267948966) on bit 1, fully controlled
X on bit 1
Ry(1.9106332362490184) on bit 0, fully controlled
X on bit 1
Rz(4.712388980384691) on bit 1, fully controlled
Ry(1.5707963267948961) on bit 1, fully controlled
Rz(1.570796326794896) on bit 1, fully controlled
Rz(1.5707963267948972) on bit 0, fully controlled
Ry(3.141592653589793) on bit 0, fully controlled
Rz(-1.5707963267948972) on bit 0, fully controlled


This can be represented by a quantum circuit (made with [Q-cirquit](http://physics.unm.edu/CQuIC/Qcircuit/)):

<img src="res/circuit1.png">

This is how you can view decomposition of matrix into 2-level gates, which is used to build sequence of gates.

In [4]:
print('\n'.join(map(str,qd.two_level_decompose_gray(A))))

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]] on (2, 3)
[[ 0.70710678-0.00000000e+00j  0.70710678-8.65956056e-17j]
 [-0.70710678-8.65956056e-17j  0.70710678-0.00000000e+00j]] on (1, 3)
[[ 0.57735027-0.00000000e+00j  0.81649658-9.99919924e-17j]
 [-0.81649658-9.99919924e-17j  0.57735027-0.00000000e+00j]] on (0, 1)
[[-7.07106781e-01+8.65956056e-17j -3.57316295e-16-7.07106781e-01j]
 [ 3.57316295e-16-7.07106781e-01j -7.07106781e-01-8.65956056e-17j]] on (1, 3)
[[ 0.00000000e+00+0.j -5.74151014e-16-1.j]
 [ 0.00000000e+00-1.j  0.00000000e+00+0.j]] on (2, 3)


Those matrices are ordered in order they are applied, so to write them as a matrix product, we have to reverse them. This product can be written as follows:
    
$$A = 
\begin{pmatrix} 0 & -i \\ -i & 0 \end{pmatrix}_{2,3}
\begin{pmatrix} -\frac{\sqrt{2}}{2} & -\frac{\sqrt{2}}{2}i \\ -\frac{\sqrt{2}}{2}i & -\frac{\sqrt{2}}{2} \end{pmatrix}_{1,3}
\begin{pmatrix} \sqrt{\frac{1}{3}} & \sqrt{\frac{2}{3}} \\ -\sqrt{\frac{2}{3}} & \sqrt{\frac{1}{3}} \end{pmatrix}_{0,1}
\begin{pmatrix} \frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} \\ -\frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} \end{pmatrix}_{1,3}
\begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}_{2,3}
$$

Or, in full form:

$$A = 
\begin{pmatrix} 1 & 0 & 0 & 0 \\0& 1 & 0& 0 \\ 0 & 0 & 0 & -i \\ 0 & 0 & -i & 0 \end{pmatrix}
\begin{pmatrix} 1 & 0 & 0 & 0 \\
                0 & -\frac{\sqrt{2}}{2} & 0 & -\frac{\sqrt{2}}{2}i \\ 
                0 & 0 & 1 & 0 \\
                0 & -\frac{\sqrt{2}}{2}i & 0 & -\frac{\sqrt{2}}{2}  \end{pmatrix}
\begin{pmatrix} \sqrt{\frac{1}{3}} & \sqrt{\frac{2}{3}} & 0 & 0 \\
                -\sqrt{\frac{2}{3}} & \sqrt{\frac{1}{3}} & 0 & 0 \\ 
                0 & 0 & 1 & 0 \\
                0 & 0 & 0 & 1 \end{pmatrix}
\begin{pmatrix} 1 & 0 & 0 & 0 \\
                0 & \frac{\sqrt{2}}{2} & 0 &  \frac{\sqrt{2}}{2} \\ 
                0 & 0 & 1 & 0 \\
                0 & -\frac{\sqrt{2}}{2} & 0 & \frac{\sqrt{2}}{2} \end{pmatrix}
\begin{pmatrix} 1 & 0 & 0 & 0 \\
                0 & 1 & 0 & 0 \\ 
                0 & 0 & 0 & 1 \\
                0 & 0 & 1 & 0 \end{pmatrix}
$$


## Output size

Number of Q# commands this tool produces is proportional to number of elements in matrix, which is $O(4^n)$, where $n$ is number of qubits in a register. More accurately, it's asymtotically $2 \cdot 4^n$. As it grows very fast, unfortunately this tool is useful only for small values of $n$.

See detailed experimental complexity analysis of this tool in [this notebook](complexity.ipynb).

## Implementation

Implementation is based on:

* Article ["Decomposition of unitary matrices and quantum gates"](https://arxiv.org/pdf/1210.7366.pdf) by Chi-Kwong Li and Rebecca Roberts;
* Book "Quantum Computing: From Linear Algebra to Physical Implementations" (chapter 4) by Mikio Nakahara and Tetsuo Ohmi.

It consists of following steps:

1. Decomposing matrix into 2-level unitary matrices;
2. Using Gray code to transform those matrices into matrices acting on states whose index differ only in one bit;
3. Implementing those matrices as fully controled single-qubit gates;
4. Implementing single-gate qubits as Rx, Ry and R1 gates;
5. Optimizations: cancelling X gates and removing identity gates.

## Paper

Algorithm used in this tool is in detail outlined in this [paper](res/Fedoriaka2019Decomposition.pdf).